# Overview
This notebook contains my work for acquisition and preparation of the scraped dyno run data.

# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import re

import wrangle

# Car Info CSV work
Let's look at car_info.csv first.

In [2]:
# ingest car info csv
info = pd.read_csv('car_info.csv')
info

,Run,Date,Car,Name,Specs
0,4,11-08-2009 05:51 pm,2009 Nissan GT-R,GOTO:Racing,"Stock engine internals, AMS turbo upgrade, Dea..."
1,5,10-21-2009 01:08 pm,2002 Subaru Impreza WRX,Neil Bywater,"2.5L Sti longblock (No avcs)FP Green, APS 525 ..."
2,31,11-12-2009 10:56 am,2010 Nissan GT-R,Jason McCartney,2010 GTR - Completely stock - 100 octane fuel ...
3,32,11-12-2009 10:57 am,2010 Nissan GT-R,Mike Cheng,2010 GTR - stock with high flow downpipe - 94 ...
4,33,11-02-2009 10:58 am,2009 Nissan GT-R,Dave Pickering,2009 GTR - stage 2 with full exhaust and stock...
...,...,...,...,...,...
4950,5926,10-13-2017 12:25 pm,2016 Volkswagen Golf R,COBB Tuning,Stage 1
4951,5927,10-13-2017 12:25 pm,2016 Volkswagen Golf R,COBB Tuning,Stage 2
4952,5933,10-13-2017 12:47 pm,2016 Volkswagen Golf R,COBB Tuning,Stock
4953,5934,10-13-2017 12:47 pm,2016 Volkswagen Golf R,COBB Tuning,Stage 1 High Boost


In [3]:
# check nulls
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4955 entries, 0 to 4954
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Run     4955 non-null   int64 
 1   Date    4955 non-null   object
 2   Car     4955 non-null   object
 3   Name    4955 non-null   object
 4   Specs   4955 non-null   object
dtypes: int64(1), object(4)
memory usage: 193.7+ KB


**Looks good, we will need to do a few things before Explore:**
- Split the Car column into year, make, and model columns
- Convert the Date column to a datetime format
- Split the runs into Train, Validate, and Test

## 'Car' column split into year, make, and model columns

In [4]:
# checking how we could split the Car column
print(info.Car.unique()[:10].tolist())

['2009 Nissan GT-R', '2002 Subaru Impreza WRX', '2010 Nissan GT-R', '2004 Subaru Impreza WRX STI', '2010 Mazda Mazdaspeed3', '2006 Subaru Impreza WRX STI', '2009 Honda Civic Si', '2007 Mazda Mazdaspeed3', '2005 Subaru Outback XT', '1998 Subaru Impreza RS']


In [5]:
# using first-4 characters as year
info['car_year'] = info.Car.str[:4]
info.car_year.sample(3, random_state=1)

1271    2010
2360    2008
2369    2011
Name: car_year, dtype: object

In [6]:
# splitting the rest of the string into make and model
make_model = info.Car.str.extract(r'\W(.*?)\W(.*?)$')
# using the second word as the make
info['car_make'] = make_model[0]
# using the last portion as the model
info['car_model'] = make_model[1]

# check work
info.head(3)

,Run,Date,Car,Name,Specs,car_year,car_make,car_model
0,4,11-08-2009 05:51 pm,2009 Nissan GT-R,GOTO:Racing,"Stock engine internals, AMS turbo upgrade, Dea...",2009,Nissan,GT-R
1,5,10-21-2009 01:08 pm,2002 Subaru Impreza WRX,Neil Bywater,"2.5L Sti longblock (No avcs)FP Green, APS 525 ...",2002,Subaru,Impreza WRX
2,31,11-12-2009 10:56 am,2010 Nissan GT-R,Jason McCartney,2010 GTR - Completely stock - 100 octane fuel ...,2010,Nissan,GT-R


In [7]:
# drop redundant Car column
info = info.drop(columns='Car')
info.head(3)

,Run,Date,Name,Specs,car_year,car_make,car_model
0,4,11-08-2009 05:51 pm,GOTO:Racing,"Stock engine internals, AMS turbo upgrade, Dea...",2009,Nissan,GT-R
1,5,10-21-2009 01:08 pm,Neil Bywater,"2.5L Sti longblock (No avcs)FP Green, APS 525 ...",2002,Subaru,Impreza WRX
2,31,11-12-2009 10:56 am,Jason McCartney,2010 GTR - Completely stock - 100 octane fuel ...,2010,Nissan,GT-R


## Datetime column

In [8]:
# capture date and time into columns
date_time = info.Date.str.extract(r'^(.*?) (.*?)$')

In [9]:
# show discrepancy in date
date_time[0].unique().tolist()[53:58]

['10-27-2009', '11-05-2009', '10-26-2009', '11-30--0001', '12-01-2009']

Runs are chronological from what I can tell, I bet we can fix the discrepancy with the correct year...

In [10]:
# check runs where with the incorrect year
info[info.Date.str.contains('11-30--0001')]

,Run,Date,Name,Specs,car_year,car_make,car_model
98,160,11-30--0001 04:00 pm,Joey Hauser,98 Octane,2008,Mitsubishi,EVO X
117,180,11-30--0001 04:00 pm,Jeff Sponaugle,"Built 2.5L, GT30R .63 Rotated Kit, 92 Octane",2008,Subaru,Impreza WRX STI
118,181,11-30--0001 04:00 pm,Jeff Sponaugle,"Built H6, Custom GT35R Kit, 92 Octane",2002,Subaru,Impreza WRX
119,182,11-30--0001 04:00 pm,Jeff Sponaugle,"Built H6, Custom GT35R Kit, 100 Octane",2002,Subaru,Impreza WRX
120,183,11-30--0001 04:00 pm,Tim Bailey,"Built 2.5L, Cosworth Heads & Cams, GT35R, AEM ...",2004,Subaru,Impreza WRX STI
216,299,11-30--0001 04:00 pm,Sage Merrill,"Evo3 16g, TMIC, 750cc Injectors, SF Intake, 18...",2002,Subaru,Impreza WRX
217,300,11-30--0001 04:00 pm,Sage Merrill,"Evo3 16g, TMIC, 750cc Injectors, SF Intake, 16...",2002,Subaru,Impreza WRX
1338,1625,11-30--0001 03:35 pm,Mark Dickins,Stage 2 19psi 91 Octane,2007,Subaru,Impreza WRX STI
1339,1626,11-30--0001 04:34 pm,Mark Dickins,Stage 2 21psi E85,2007,Subaru,Impreza WRX STI
1404,1695,11-30--0001 04:41 pm,RUF RUF,Stock Baseline,2007,Porsche,911


In [11]:
def run_date_fixer(col):
    """ Pass back the previous and next run numbers for erroneous run date runs, if available """
    col_prev = col - 1
    col_next = col + 1
    new_col = col.append(col_prev).append(col_next).tolist()
    return new_col

In [12]:
# get list of runs
run_checks = run_date_fixer(info[info.Date.str.contains('11-30--0001')].Run)

# check work
run_checks.sort()
print(run_checks)

[159, 160, 161, 179, 180, 180, 181, 181, 181, 182, 182, 182, 183, 183, 184, 298, 299, 299, 300, 300, 301, 1624, 1625, 1625, 1626, 1626, 1627, 1694, 1695, 1696, 1696, 1697, 1698, 2502, 2503, 2504, 2882, 2883, 2883, 2884, 2884, 2885, 3402, 3403, 3403, 3404, 3404, 3404, 3405, 3405, 3405, 3406, 3406, 3406, 3407, 3407, 3407, 3408, 3408, 3408, 3409, 3409, 3410, 4132, 4133, 4133, 4134, 4134, 4134, 4135, 4135, 4135, 4136, 4136, 4137, 4298, 4299, 4299, 4300, 4300, 4300, 4301, 4301, 4301, 4302, 4302, 4303]


In [13]:
# use run_checks list to pass back rows with that run number
info[info.Run.isin(run_checks[:20])]

,Run,Date,Name,Specs,car_year,car_make,car_model
97,159,10-26-2009 11:38 am,Jake Zvirzdys,"Stage 2, 92 Octane",2006,Subaru,Impreza WRX
98,160,11-30--0001 04:00 pm,Joey Hauser,98 Octane,2008,Mitsubishi,EVO X
99,161,12-07-2009 11:42 am,John Wulf,"Stage 2, AEM Intake, AVO TMIC, 17.5psi, 92 Octane",2006,Subaru,Legacy 2.5GT
116,179,10-08-2009 03:38 pm,Tad Ogland,"Stage 2, 92 Octane",2008,Mitsubishi,EVO X GSR
117,180,11-30--0001 04:00 pm,Jeff Sponaugle,"Built 2.5L, GT30R .63 Rotated Kit, 92 Octane",2008,Subaru,Impreza WRX STI
118,181,11-30--0001 04:00 pm,Jeff Sponaugle,"Built H6, Custom GT35R Kit, 92 Octane",2002,Subaru,Impreza WRX
119,182,11-30--0001 04:00 pm,Jeff Sponaugle,"Built H6, Custom GT35R Kit, 100 Octane",2002,Subaru,Impreza WRX
120,183,11-30--0001 04:00 pm,Tim Bailey,"Built 2.5L, Cosworth Heads & Cams, GT35R, AEM ...",2004,Subaru,Impreza WRX STI
121,184,11-30-2009 03:46 pm,Tim Diens,"Stock, 92 Octane",2009,Subaru,Impreza WRX STI
215,298,01-01-2010 05:08 pm,Lance Lucas,"GT35R .82, Built 2.5L Longblock, 272 Cams, 21-...",2004,Subaru,Impreza WRX


Based on the need to clean 'Date' column and the lack of value it provides, I'll just drop the 'Date' column.

In [14]:
# drop Date column
info = info.drop(columns='Date')
info.head(3)

,Run,Name,Specs,car_year,car_make,car_model
0,4,GOTO:Racing,"Stock engine internals, AMS turbo upgrade, Dea...",2009,Nissan,GT-R
1,5,Neil Bywater,"2.5L Sti longblock (No avcs)FP Green, APS 525 ...",2002,Subaru,Impreza WRX
2,31,Jason McCartney,2010 GTR - Completely stock - 100 octane fuel ...,2010,Nissan,GT-R


Looks good- we'll split our data after we look into dyno_runs.csv next.

# Dyno Runs CSV work
Now let's look at dyno_runs.csv.

In [15]:
# ingest run data
runs = pd.read_csv('dyno_runs.csv', index_col=0)
runs

,Run,RPM,HP,Torque,AFR,Boost
0,4,2200.0,106.10,252.50,7.47,4.12
1,4,2300.0,145.80,333.10,7.48,5.43
2,4,2400.0,177.90,389.50,7.48,6.78
3,4,2500.0,202.30,424.90,7.48,8.15
4,4,2600.0,221.80,447.90,7.48,9.55
...,...,...,...,...,...,...
2044935,5951,3.0,151.91,316.87,NaN,NaN
2044936,5951,4.0,253.33,380.61,NaN,NaN
2044937,5951,5.0,300.56,349.22,NaN,NaN
2044938,5951,6.0,305.76,292.02,NaN,NaN


In [16]:
# check nulls
runs.isna().sum()

Run            0
RPM         7214
HP             0
Torque         0
AFR       199729
Boost     197572
dtype: int64

In [17]:
# figure out which runs have nulls in RPM column
null_RPM_runs = runs[runs.RPM.isna()].Run.unique().tolist()
print(null_RPM_runs)

[978, 979, 1578, 3266, 3454, 3498, 3631, 3632, 3682, 4022, 4023, 4330, 4331, 4652, 4670, 4671, 4672, 5025, 5380, 5403, 5407, 5434]


In [18]:
# check if all RPM values are null in these runs, or just partially null
runs[runs.Run.isin(null_RPM_runs)].RPM.isna().sum()

7214

All RPM values are null in these runs. Let's see if there's a clue in car_info.csv as to why.

In [19]:
# check car info for runs with nulls in RPM
info[info.Run.isin(null_RPM_runs)]

,Run,Name,Specs,car_year,car_make,car_model
3102,3682,David Palmer,"91 oct, COBB downpipe, COBB AP+PRoTUne",2010,Subaru,Forester


Okay, it seems these runs don't exist in car_info.csv. Only one run does. We will therefore drop all rows with nulls in the RPM column.

In [20]:
# drop nulls
runs = runs[runs.RPM.notna()].reset_index(drop=True)

Now let's see if dropping the runs with nulls in AFR or Boost is fine. 

In [21]:
# check percentage of rows with nulls in either AFR or Boost columns
print(str(int((runs.AFR.isna() | runs.Boost.isna()).sum() / runs.shape[0] * 100)) + '%')

15%


In [22]:
# is the Boost column less than 15%?
print(str(int(runs.Boost.isna().sum() / runs.shape[0] * 100)) + '%')

9%


So, I'm going to drop the 'AFR' column and drop rows with nulls in the 'Boost' column. Let me explain why.

**Why I'm dropping the 'AFR' column:**
1. A balance in air-fuel ratio (AFR) provides the most horsepower, and this ratio is manually adjusted
2. This balance point is different for different cars and setups
3. The car variety in this dataset is too diverse to consider raw values in AFR
4. Analysis of AFR on a car-by-car basis is possible, but not valuable to a general audience owning various vehicles

**Why I'm only dropping nulls in the Boost column:**
1. In nearly all cases, as boost increases, power increases
2. The few potential exceptions to this relationship will likely be explained by the parts in the 'Specs' column
3. Raw increases in 'Boost' values will show increases in horsepower, so it's valuable to keep this as a feature

In [23]:
# dropping AFR column
runs = runs.drop(columns = 'AFR')

# dropping rows with nulls in Boost column
runs = runs[runs.Boost.notna()]

# check work
runs.isna().sum()

Run       0
RPM       0
HP        0
Torque    0
Boost     0
dtype: int64

Looks good, I'll push these changes to wrangle.py now.

Checking if the changes worked...

In [24]:
# acquire, clean, and split data using script
info_train, runs_train = wrangle.prep_explore_MVP()

In [25]:
# check if run numbers in train split are the same
info_train.run.sort_values().unique().tolist() == runs_train.run.sort_values().unique().tolist()

True

# Exploration Re-Wrangle
In Explore I noticed more prep that needs to be done with the data, below captures that work.

In [26]:
pd.set_option('display.max_colwidth', None)
info = info.reset_index(drop=True)
info

,Run,Name,Specs,car_year,car_make,car_model
0,4,GOTO:Racing,"Stock engine internals, AMS turbo upgrade, Deatschwerks 800cc injectors, full catless 3"" open exhaust, Harman Motive custom 3.1"" ID intake with stock MAF sensors, 100 octane unleaded fuel",2009,Nissan,GT-R
1,5,Neil Bywater,"2.5L Sti longblock (No avcs)FP Green, APS 525 FMIC, APS 65mm intake Cobb turbo-back, APS turbo inlet, DW 850 injectors",2002,Subaru,Impreza WRX
2,31,Jason McCartney,2010 GTR - Completely stock - 100 octane fuel - Cobb stage 1 v110 100 octane beta,2010,Nissan,GT-R
3,32,Mike Cheng,2010 GTR - stock with high flow downpipe - 94 octane - COBB v110 beta stage 2 tune,2010,Nissan,GT-R
4,33,Dave Pickering,2009 GTR - stage 2 with full exhaust and stock downpipes - cobb v110 stage 2 92 octane beta,2009,Nissan,GT-R
...,...,...,...,...,...,...
4950,5926,COBB Tuning,Stage 1,2016,Volkswagen,Golf R
4951,5927,COBB Tuning,Stage 2,2016,Volkswagen,Golf R
4952,5933,COBB Tuning,Stock,2016,Volkswagen,Golf R
4953,5934,COBB Tuning,Stage 1 High Boost,2016,Volkswagen,Golf R


In [27]:
def check_non_keywords(df, col):
    """ Check value counts of words I did not designate as keywords in product names """
    # check all rows without keywords for each unique word's value counts in entire list
    print(
        pd.Series( # make a Series of each instance of each word
            ' '.join(
                    df[~df.has_keyword]    # look at rows we haven't caught with a keyword yet
                    [col].tolist()        # put all 'name' cells in a list
                    ).split()        # join all lists into one string, then split the string into a list of each word
        ).value_counts()        # calculate the value counts of each word in the series
        .head(20)         # display the top 10 (changed from 30 to 10 after the words I wanted were captured)
    )

In [28]:
info['has_keyword'] = False

In [29]:
check_non_keywords(info, 'Specs')

COBB           3437
Tuning         2859
Octane         2384
92             1627
AccessPORT,    1241
Perrin         1044
93             1002
Stage           899
-               880
Intake,         832
SF              698
Fuel            625
Injectors,      597
2               583
FMIC,           491
3               464
Catless         460
Turboback,      458
Downpipe,       446
E85             421
dtype: int64


## PSI

In [30]:
# capture nearly all psi ratings
info['psi'] = info.Specs.str.extract(r'^.*\s(\d+\.*\d*)\s?[Pp][Ss][Ii].*$')

In [31]:
# fix '(number) Peak PSI' issue
newthing = info.Specs.str.extract(r'^.*\b(\d\d\.\d) Peak PSI.*$')
indices = newthing[newthing[0].notna()].index
for ind in indices:
    info.loc[ind, 'psi'] = newthing[0][ind]

In [32]:
# check if 'Peak PSI' has a number in the 'psi' column now
info[info.Specs.str.contains('Peak PSI')]

,Run,Name,Specs,car_year,car_make,car_model,has_keyword,psi
731,928,COBB Tuning,Stage 2 OTS Mapping- 17.5 Peak PSI- 93 Octane,2010,Subaru,Legacy 2.5GT,False,17.5
732,929,COBB Tuning,Stage 2 OTS Mapping- 14.5 Peak PSI- 91 Octane,2010,Subaru,Legacy 2.5GT,False,14.5


In [33]:
# indicate in new column if Specs has 'psi' in it
info['has_psi'] = info.Specs.str.contains('PSI|psi')

In [34]:
# check all occurences where 'psi' is null but Specs does have a 'psi'
info[(info.has_psi) & (info.psi.isna())][['Specs','psi','has_psi']]

,Specs,psi,has_psi
215,"GT35R .82, Built 2.5L Longblock, 272 Cams, 21-19psi Boost, E85",NaN,True
241,18psi w/E85,NaN,True
242,8.5psi w/E85,NaN,True
243,16psi w/E85,NaN,True
330,"27 psi 93 octane -FP Red with Forge WG Act., DW 800, AMS Downpipe, AMS Intercooler and Pipes, Agency Power Intake, Prodrive 3Port",NaN,True
...,...,...,...
4860,"15psi - E70 - COBB Accessport w/ COBB Flex Fuel Kit, COBB Turboback, COBB EWG- Rerouted, Process West TMIC, ID 1000cc Injectors, AEM Fuel Pump.",NaN,True
4861,"17psi - E70 - COBB Accessport w/ COBB Flex Fuel Kit, COBB Turboback, COBB EWG- Rerouted, Process West TMIC, ID 1000cc Injectors, AEM Fuel Pump.",NaN,True
4862,"19psi - E70 - COBB Accessport w/ COBB Flex Fuel Kit, COBB Turboback, COBB EWG- Rerouted, Process West TMIC, ID 1000cc Injectors, AEM Fuel Pump.",NaN,True
4875,"18psi - COBB SF Intake/Airbox, Perrin TMIC, FIC 1100cc Injectors, Fuel Pump, AMR Big Tube Header, Turbosmart EWG, and Invidia TBE.",NaN,True


Looks good for PSI, let's do fuel octane now.

## Octane

In [35]:
info['octane'] = info.Specs.str.extract(r'^.*\b(\d+)[,\s]?\s?[Oo][Cc][Tt].*$')
info['has_octane'] = info.Specs.str.contains('Octane|octane')
info[(info.has_octane) & (info.octane.isna())][['Specs','octane','has_octane']]

,Specs,octane,has_octane
580,FP Green 20psi E85 Octane Sport#,NaN,True
2412,"COBB Socal Tuned ""Sport"" map, HKS DP, Midpipe and CBE. GotBoost 3"" intakes. 91 CA Octane :-(",NaN,True
2413,"COBB Socal Tuned ""Mild"" map, HKS DP, Midpipe and CBE. GotBoost 3"" intakes. 91 CA Octane :-(",NaN,True
2414,"SP Eng map, HKS DP, Midpipe and CBE, Panel filters. 91 CA Octane :-(",NaN,True
4460,"ANC91 Octane - Added Blouch Dom1.5XTR Turbo, COBB 1300s",NaN,True
4905,ByDesign StageIV - ACN91 Octane,NaN,True


In [36]:
# octane overall capture
info['octane'] = info.Specs.str.extract(r'^.*\b(\d+)[,\s]?\s?[Oo][Cc][Tt].*$')
# 93 octane fuel
octane_93_indices = info[info.Specs.str.contains(' 93 ')].index
for ind in octane_93_indices:
    info.loc[ind, 'octane'] = 93
# 91 octane fuel
octane_91_indices = info[info.Specs.str.contains('ACN91|ANC91|91 CA| 91 ')].index
for ind in octane_91_indices:
    info.loc[ind, 'octane'] = 91
# 104 octane fuel
octane_104_indices = info[info.Specs.str.contains('104')].index
for ind in octane_104_indices:
    info.loc[ind, 'octane'] = 104
# e85 fuel
e85_indices = info[info.Specs.str.contains('E85|E-85')].index
for ind in e85_indices:
    info.loc[ind, 'octane'] = 105
# MS109 fuel
ms109_indices = info[info.Specs.str.contains('MS109')].index
for ind in ms109_indices:
    info.loc[ind, 'octane'] = 109

In [37]:
info[(info.has_octane) & (info.octane.isna())][['Specs','octane','has_octane']]

,Specs,octane,has_octane


Looks good for Octane, let's move on to the AccessPORT, which is a modification to the ECU. In layman's terms it adjusts the car's computer so that the car performs better.

In [38]:
# drop unnecessary columns
info = info.drop(columns=['has_psi','has_octane'])

# Post-MVP
## Add Stock Horsepower Information for 'car_model' Values

In [39]:
# create merge-ready dataframe of stock horsepower values
hp_dict = wrangle.horsepower_dict()
stock_hp_df = pd.DataFrame(hp_dict, index=['stock_hp']).T.reset_index().rename(columns={'index':'car_model'})

In [40]:
# add stock_hp column to car_info dataframe
info['car_model'] = info['car_year'].astype('str') + ' ' + info['car_model']
info = pd.merge(left=info, right=stock_hp_df, on='car_model')

In [41]:
info.head(3)

,Run,Name,Specs,car_year,car_make,car_model,has_keyword,psi,octane,stock_hp
0,4,GOTO:Racing,"Stock engine internals, AMS turbo upgrade, Deatschwerks 800cc injectors, full catless 3"" open exhaust, Harman Motive custom 3.1"" ID intake with stock MAF sensors, 100 octane unleaded fuel",2009,Nissan,2009 GT-R,False,NaN,100,480
1,33,Dave Pickering,2009 GTR - stage 2 with full exhaust and stock downpipes - cobb v110 stage 2 92 octane beta,2009,Nissan,2009 GT-R,False,NaN,92,480
2,54,Tim Bailey,2009 GTR stage 2 91 octane,2009,Nissan,2009 GT-R,False,NaN,91,480


## Imputing PSI Using Max 'boost' Value From Run

In [42]:
# group runs by max boost value
max_boost_df = pd.DataFrame(runs.groupby('Run').Boost.max()).reset_index()
# merge with car_info
info = pd.merge(left=info, right=max_boost_df, on='Run')
# check if psi and Boost are similar for non-null psi values
info[['psi','Boost']].sample(10, random_state=1)

,psi,Boost
453,20,20.70
364,17.5,17.00
372,18,18.28
1637,13,13.10
857,16,15.70
2204,18,18.10
2318,NaN,21.10
3054,NaN,28.40
3774,18,17.80
1647,16.5,16.92


Looks like we're safe to impute based on the run's max boost PSI value.

In [43]:
# fill nulls with max boost PSI value for run
info['psi'] = info['psi'].fillna((info.Boost * 2).astype('int') / 2) # keep .5 precision
# drop redundant 'Boost' column
info = info.drop(columns='Boost')

In [44]:
# check work
info.head()

,Run,Name,Specs,car_year,car_make,car_model,has_keyword,psi,octane,stock_hp
0,4,GOTO:Racing,"Stock engine internals, AMS turbo upgrade, Deatschwerks 800cc injectors, full catless 3"" open exhaust, Harman Motive custom 3.1"" ID intake with stock MAF sensors, 100 octane unleaded fuel",2009,Nissan,2009 GT-R,False,22.0,100,480
1,33,Dave Pickering,2009 GTR - stage 2 with full exhaust and stock downpipes - cobb v110 stage 2 92 octane beta,2009,Nissan,2009 GT-R,False,17.5,92,480
2,54,Tim Bailey,2009 GTR stage 2 91 octane,2009,Nissan,2009 GT-R,False,17.0,91,480
3,55,Tim Bailey,2009 GTR stage 2 93 octane,2009,Nissan,2009 GT-R,False,17.0,93,480
4,66,Samuel Frankel,Cat Back Ext. Stage 2 19 psi,2009,Nissan,2009 GT-R,False,19,NaN,480


## Imputing Nulls in 'octane' Using Most Common Octane

In [45]:
# get most common fuel octane
info.octane.value_counts().head(3)

92     1596
93      854
105     467
Name: octane, dtype: int64

In [46]:
# fill octane nulls with 92
info['octane'] = info['octane'].fillna(92).astype('int')
# check work
info.sample(5, random_state=1)

,Run,Name,Specs,car_year,car_make,car_model,has_keyword,psi,octane,stock_hp
453,2148,Peter Cowan,"COBB Tuning AccessPORT, GT2860 .63 A/R, Deatschwerks 850cc Injectors, Perrin Exhaust Manifold, Perrin Big MAF Intake, Catted Midpipe- 20psi- Pump Fuel",2004,Subaru,2004 Impreza WRX STI,False,20,92,300
364,4370,Reilley Friessen,"COBB Tuning Accessport, Downpipes, Midpipe, HKS Catback, GotBoost 3"" Intakes, SIR 750cc Injectors, '12 Inlets- 17.5psi- 92 Octane",2010,Nissan,2010 GT-R,False,17.5,92,485
372,5128,Joshua Munoz,Stage 3 18psi 93 Octane,2010,Nissan,2010 GT-R,False,18,93,485
1637,1892,Russ Hoefer,"COBB Tuning AccessPORT, COBB Tuning SF Intake, Catted Downpipe- 13psi- 92 Octane",2006,Subaru,2006 Impreza WRX,False,13,92,230
857,811,Brett Ensor,"COBB Tuning AccessPORT, Perrin Catted Turboback Exhaust, AEM Intake- 16psi- 92 Octane",2008,Subaru,2008 Impreza WRX STI,False,16,92,305


## AccessPORT, OTS, ProTune, and Stages
**The AccessPORT is a product that COBB Tuning offers to modify the engine control computer.** The modifications are advertised as increasing performance for a variety of cars using Off-The-Shelf (OTS) configurations. Anytime 'Access', 'OTS', or 'ProTune' is mentioned, the car's computer has been modified. Because this is an offer from COBB Tuning, we can expect it will generally be noted in car_info 'Specs' column.

In addition to 'Access' and 'OTS', any mention of 'Stage' indicates a COBB Tuning powertrain modification that generally requires an AccessPORT/OTS to adjust the engine control computer to perform best with the new parts.

We will create a feature for these words indicating that the engine control computer has been modified.

In [47]:
# create feature
tuned_cpu_keywords = ['Access','OTS','Stage','stage','Map','map','ProTune','Protune', 
                      'Pro-tune', 'ProTUNE', 'COBB AP', 'Cobb AP']
info['tuned_cpu'] = info.Specs.str.contains('|'.join(tuned_cpu_keywords))
# check work
info.sample(7, random_state=2)

,Run,Name,Specs,car_year,car_make,car_model,has_keyword,psi,octane,stock_hp,tuned_cpu
3623,2952,Mohamed Alsabah,Alpha 9 95 octane map,2012,Nissan,2012 GT-R,False,21.0,95,530,True
2079,1017,Kyle White,"COBB Tuning AccessPORT, GTSpec Exhaust Manifold/Up Pipe, Cobb Downpipe, Cobb SF Intake, Catback- 17psi- 92 Octane",2006,Subaru,2006 Legacy 2.5GT,False,17,92,250,True
4110,4842,Joseph R,Invidia Downpipe,2013,Subaru,2013 Impreza WRX,False,17.5,92,265,False
1966,757,Daniel Fisher,"COBB Tuning AccessPORT, SPT Catback, COBB Tuning Downpipe, Deatschwerks 1000cc Injectors, Walbro Fuel Pump, K&N Typhoon Intake, Perrin TMIC, Tial 44mm EWG- 18.5psi- E85",2009,Subaru,2009 Impreza WRX,False,18.5,105,265,True
2198,2232,Aaron Singleton,"COBB Tuning AccessPORT, TurboXS FMIC, Catless Turboback, DOM 3 w/3"" Inlet, 38mm EWG, KS 73mm Intake, TGV Deletes, Deatschwerks 1300cc Injectors, Manual Boost Controller - 23psi- 92 Octane",2007,Subaru,2007 Impreza WRX STI,False,23,92,300,True
1364,384,Kyle Schmidt,"Blouch TD06-20G, DW850cc Injectors, Walbro Fuel Pump, GrimmSpeed Manifold/Up Pipe/44mm Wastegate Kit, Perrin FMIC, Perrin Big MAF- 19psi- 92 Octane",2005,Subaru,2005 Impreza WRX STI,False,19,92,300,False
313,4858,Jeremy G,"VF22, TMIC, Sti Pinks, Perring TIH, Perrin Intake, TBE",2002,Subaru,2002 Impreza WRX,False,17.0,92,227,False


## Final Thoughts on Feature Engineering the 'Specs' Column
### Stopping with above features due to lack of information
So I've created a few features indicating boost PSI, fuel octane, stock horsepower, and whether the engine control computer has been modified or not. Because of the nature of the *hand-entered* 'Specs' values, additional feature engineering would not be reliable. 
1. We can't say for sure that one car has something, and another car does not. 
2. Specific modifications are likely to have low overall counts, making the feature less useful for prediction.

### Potential for owner-by-owner analysis
Now, what may be valuable in a second post-MVP iteration is to look at cars that have been run multiple times with different setups. We know that certain cars are run multiple times with different setups thanks to the 'Name' column. 
1. Comparing one person's car across multiple different runs may indicate that certain parts improve performance.
2. This analysis may prove valuable as an insight, but would not be valuable for prediction.

In [48]:
# check prep_explore function
info_train, runs_train = wrangle.prep_explore()

In [49]:
info.head()

,Run,Name,Specs,car_year,car_make,car_model,has_keyword,psi,octane,stock_hp,tuned_cpu
0,4,GOTO:Racing,"Stock engine internals, AMS turbo upgrade, Deatschwerks 800cc injectors, full catless 3"" open exhaust, Harman Motive custom 3.1"" ID intake with stock MAF sensors, 100 octane unleaded fuel",2009,Nissan,2009 GT-R,False,22.0,100,480,False
1,33,Dave Pickering,2009 GTR - stage 2 with full exhaust and stock downpipes - cobb v110 stage 2 92 octane beta,2009,Nissan,2009 GT-R,False,17.5,92,480,True
2,54,Tim Bailey,2009 GTR stage 2 91 octane,2009,Nissan,2009 GT-R,False,17.0,91,480,True
3,55,Tim Bailey,2009 GTR stage 2 93 octane,2009,Nissan,2009 GT-R,False,17.0,93,480,True
4,66,Samuel Frankel,Cat Back Ext. Stage 2 19 psi,2009,Nissan,2009 GT-R,False,19,92,480,True


In [50]:
runs.head()

,Run,RPM,HP,Torque,Boost
0,4,2200.0,106.1,252.5,4.12
1,4,2300.0,145.8,333.1,5.43
2,4,2400.0,177.9,389.5,6.78
3,4,2500.0,202.3,424.9,8.15
4,4,2600.0,221.8,447.9,9.55
